In [14]:
# load csv file from gzip
import gzip
import pandas as pd
import os
import numpy as np

file_path = '0_OptionMetrics_Option_Volume_2012_2023_08.gz'

# load csv file from gzip
def load_csv_gzip(file_path, **kwargs):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File {file_path} does not exist.")
    with gzip.open(file_path, 'rt', encoding='utf-8') as f:
        df = pd.read_csv(f, **kwargs)
    return df


# load the data
df = load_csv_gzip(file_path, low_memory=False)

# check the data
print(df.head())


# Ensure the columns are correct
df['date'] = pd.to_datetime(df['date'])

# Convert 'ticker' to string and strip whitespace
df['cp_flag'] = df['cp_flag'].astype(str).str.strip()

# Keep only relevant columns and filter for call and put options
df = df[df['cp_flag'].isin(['C', 'P'])]
# Pivot table
volume_pivot = df.pivot_table(index=['date', 'ticker'], 
                              columns='cp_flag', 
                              values='volume', 
                              aggfunc='sum', 
                              fill_value=0)

# Flatten MultiIndex columns
volume_pivot.columns = [str(col) if isinstance(col, str) else str(col[1]) for col in volume_pivot.columns]

# Rename for clarity
volume_pivot = volume_pivot.rename(columns={'C': 'CallVolume', 'P': 'PutVolume'})

# Compute ratio (avoid divide-by-zero)
volume_pivot['PutCallRatio'] = volume_pivot['PutVolume'] / volume_pivot['CallVolume'].replace(0, np.nan)
volume_pivot['PutCallRatio'] = volume_pivot['PutCallRatio'].fillna(0).round(4)

# Reset index if needed
volume_pivot = volume_pivot.reset_index()

# Keep only relevant columns
volume_pivot = volume_pivot[['date', 'ticker', 'PutCallRatio']]

# Sort by ticker and date
volume_pivot = volume_pivot.sort_values(by=['ticker', 'date'])

# Preview and save
print(volume_pivot.head())
volume_pivot.to_csv("1_1_PutCallRatio_2012_2023_08.csv", index=False)



   secid        date cp_flag  index_flag ticker  volume
0   5139  2022-03-29       P           0    CAE       3
1   5139  2022-10-24       C           0    CAE      16
2   5139  2021-03-17     NaN           0    CAE      53
3   5139  2021-11-11     NaN           0    CAE      42
4   5139  2022-05-04       C           0    CAE     133
            date ticker  PutCallRatio
0     2012-01-03      A        0.2732
2732  2012-01-04      A        1.3213
5463  2012-01-05      A        0.8842
8194  2012-01-06      A        0.9620
10928 2012-01-09      A        0.3901


In [1]:
# Sort Putcallratio downloaded from Bloomberg (0_Putcall_ratio_from_Bloomberg_2023_08_12.csv)
import pandas as pd
# load the data
df_bloomberg = pd.read_csv("0_Putcall_ratio_from_Bloomberg_2023_08_12.csv", skiprows=1, low_memory=False)

# Convert Date to year-month-day format
df_bloomberg['Date'] = pd.to_datetime(df_bloomberg['Date'], format='%d/%m/%Y')

df_long = df_bloomberg.melt(id_vars=['Date'], var_name='ticker', value_name='PutCallRatio')
df_long['ticker'] = df_long['ticker'].str.replace(" US Equity", "", regex=False)

# Only keep date from 2023-09-01
df_long = df_long.rename(columns={'Date': 'date'})
df_long = df_long[df_long['date'] >= '2023-09-01']
df_long.to_csv("1_2_PutCallRatio_2023_09_12.csv", index=False)



In [2]:
# Match with train_data_2013_2018.csv's date and tic with 1_put_call_ratio_by_date_ticker.csv, output PutCallRatio to train_data_2013_2018.csv
import pandas as pd
# Load the train data
train_data = pd.read_csv("train_data_2013_2018.csv")
# Load the volume data
volume_data = pd.read_csv("1_1_PutCallRatio_2012_2023_08.csv")

# Merge with the volume data
train_data['date'] = pd.to_datetime(train_data['date'])
train_data['tic'] = train_data['tic'].astype(str).str.strip()
volume_data['date'] = pd.to_datetime(volume_data['date'])
train_data = train_data.merge(volume_data[['date', 'ticker', 'PutCallRatio']], 
                               left_on=['date', 'tic'], 
                               right_on=['date', 'ticker'], 
                               how='left')
# output the result to train_data_2013_2018.csv
train_data = train_data.drop(columns=['ticker'])
train_data.to_csv("2_1_train_data_with_putcall_2013_2018.csv", index=False)

In [3]:
# Match with trade_data_2019_2023.csv's date and tic with 1_put_call_ratio_by_date_ticker.csv, output PutCallRatio to trade_data_2019_2023.csv
import pandas as pd
# Load the train data
train_data = pd.read_csv("trade_data_2019_2023.csv")
# Load the volume data
volume_data = pd.read_csv("1_2_PutCallRatio_2023_09_12.csv")
# Merge with the volume data
train_data['date'] = pd.to_datetime(train_data['date'])
volume_data['date'] = pd.to_datetime(volume_data['date'])
train_data['tic'] = train_data['tic'].astype(str).str.strip()
train_data = train_data.merge(volume_data[['date', 'ticker', 'PutCallRatio']], 
                               left_on=['date', 'tic'], 
                               right_on=['date', 'ticker'], 
                               how='left')
# output the result to trade_data_2019_2023.csv
train_data = train_data.drop(columns=['ticker'])
train_data.to_csv("2_2_trade_data_with_putcall_2019_2023.csv", index=False)

################

Abnormal Put Call ratio 30 days rolling window

In [11]:
import pandas as pd

# Load the train and trade data
train_data = pd.read_csv("2_1_train_data_with_putcall_2013_2018.csv")
trade_data = pd.read_csv("2_2_trade_data_with_putcall_2019_2023.csv")

# Combine the two dataframes
combined_data = pd.concat([train_data, trade_data], ignore_index=True)

# Select only the relevant columns
relevant_columns = ['vix', 'turbulence', 'PutCallRatio']
summary_stats = combined_data[relevant_columns].describe()

# Save the summary statistics to a CSV file
summary_stats.to_csv("summary_statistics.csv", index=True)
